In [2]:
%matplotlib inline
import numpy as np

import jupyter_manim
from manim import *

Manim Community v0.13.1

In [90]:
%%manim -qm -v WARNING Grover_2qbit

class Grover_2qbit(Scene):
    config= {
        "x_lim":int,    # define leftmost lowerlimit of x for circuit and gates
        "y_lim":int,    # define up-most limit of y for circuit and gates
        
    }
    def __init__(self, **kwargs):
        super(Grover_2qbit, self).__init__()       #inheritance from class Scene
        self.x_lim=-6
        self.y_lim=-1
        self.gap=2
        self.n_qubit=2
        self.sect=6
        self.x_range=[None for i in range(self.sect+1)]
        self.dots=[Dot() for i in range(self.n_qubit)]
        self.circuitline=[[None for i in range(5)] for i in range(5)]   # List to hold all circuit line elements and VGroup  to include all the Mobjects of circuitline
        self.circuit=VGroup()
    def construct(self):
        
        # Algorithm name
        algorithm="Grover's Algorithm"
        title = Title(algorithm,include_underline=False).set_y(3.1)     #title of the ALgorithm
        self.add_foreground_mobject(title)

        #section zero: create all registers: quantum and classical
        # regs=[MathTex(r"q_"+str(i)) for i in range(n_qubit)] # create quantum register
        
        registers=VGroup()
        regs=[MathTex(r"q_"+str(i)) for i in range(self.n_qubit)]
        for i,reg in enumerate(regs):       #add registers to the scene
            reg.set_x(self.x_lim).set_y(self.y_lim+(i)*self.gap)
            registers.add(reg)
        self.add(registers)

        def create_gate(gate:str,x:int,y:int,control=None,scale=1,textcolor=WHITE,boxcolor=BLUE,fontsize=48):
            if control:
                dot=Dot().set_x(x).set_y(control)
                control_line= Line([x,control,0], [x,y,0])
                gate_ = MathTex(gate,color=textcolor,font_size=fontsize).scale(scale+0.5).set_x(x).set_y(y)
                gate_.bg = BackgroundRectangle(gate_,stroke_width=1,color=boxcolor, fill_opacity=1).scale(scale+1)
                return VGroup(control_line,gate_.bg,gate_,dot)
            else:
                gate_ = MathTex(gate,color=textcolor,font_size=fontsize).scale(scale+0.5).set_x(x).set_y(y)
                gate_.bg = BackgroundRectangle(gate_,stroke_width=1,color=boxcolor, fill_opacity=1).scale(scale+1)
                return VGroup(gate_.bg,gate_)
        
        def draw_circuitline(sect:int,xrange:list,barrier=True):
            for i in range(self.n_qubit):
                self.circuitline[sect][i]=Line([xrange[0],self.y_lim+i*self.gap,-0.5], [xrange[1],self.y_lim+i*self.gap,-0.5])
                self.circuit.add(self.circuitline[sect][i])
                self.add(self.circuitline[sect][i])
            if barrier:
                barrierline=DashedLine([xrange[1],self.y_lim-0.5,0], [xrange[1],self.y_lim+self.gap*(self.n_qubit-0.5),0],dashed_ratio=0.75).set_opacity(0.5)
                self.add(barrierline)
                # self.add(VGroup(circuit,barrierline))
                # return VGroup(circuit,barrier)
        def move_dot(sect: int,dots=self.dots):
            animation=[]
            for j in range(self.n_qubit):
                animation.append(MoveAlongPath(dots[j], self.circuitline[sect][j]))
            # self.play(*animation)
            return animation
        
        def parallelgates(gate,x,sect:int,qubits=self.n_qubit):
            hgates=[]
            for i in range(qubits):
                hgate=create_gate(gate,x=x,y=self.y_lim+i*self.gap)
                hgates.append(Create(hgate))
            return hgates

        
        # Section I: Uniform Superposition
        depth=sect=1
        self.x_range[0]=[self.x_lim,self.x_lim+0.5+0.75*(depth-1)]
        self.x_range[sect]=[self.x_range[sect-1][1],self.x_range[sect-1][1]+1.5+0.5*(depth)]
        
        draw_circuitline(sect,xrange=self.x_range[sect])
        annot=MathTex(r"H^{\otimes 2}").scale(0.75).set_x(self.x_range[sect][0]+1).set_y(self.y_lim-1)
        self.play(*move_dot(1),*parallelgates(gate=r"H",sect=1,x=1.01*self.x_range[sect][0]+1),Create(annot))

        # section II: Oracle function Controlled Phase rotation
        sect=2
        depth==0
        self.x_range[sect]=[self.x_range[sect-1][1],self.x_range[sect-1][1]+1.5+0.5*(depth)]
        pgate_1=create_gate(r"Z",x=self.x_range[sect][0]+(1+0.5)*0.75,y=self.y_lim+(1+0.01)*self.gap,
                                        fontsize=48,control=self.y_lim-(0.0)*self.gap)
        draw_circuitline(sect,xrange=self.x_range[sect])
        annot=MathTex(r"Oracle").scale(0.75).set_x(self.x_range[sect][0]+1).set_y(self.y_lim-1)
        self.play(*move_dot(sect),Create(pgate_1),Create(annot))
        print(self.x_range) 
        # Section III: Reflection
        sect=3
        depth=3
        self.x_range[sect]=[self.x_range[sect-1][1],self.x_range[sect-1][1]+1.5+1*(depth+2)]
        pgate_1=create_gate(r"Z",x=self.x_range[sect][0]+3.5+0.5*0.75,y=self.y_lim+1.001*self.gap,
                                        fontsize=48,control=self.y_lim-(0.0)*self.gap)
        draw_circuitline(sect,xrange=self.x_range[sect],barrier=False)
        annot=MathTex("Reflection \; layer").scale(0.75).set_x(sum(self.x_range[sect])/len(self.x_range[sect])).set_y(self.y_lim-1)
        self.play(*move_dot(sect),*parallelgates(gate=r"H",sect=sect,x=self.x_range[sect][0]+0.5+0.5*0.75),
                                *parallelgates(gate=r"Z",sect=sect,x=self.x_range[sect][0]+2.+0.5*0.75),
                                Create(pgate_1),
                                *parallelgates(gate=r"H",sect=sect,x=self.x_range[sect][0]+5+0.5*0.75),
                                Create(annot),runtime=3)


        self.wait(2)

        

[[-6, -5.5], [-5.5, -3.5], [-3.5, -1.5], None, None, None, None]


In [81]:
%%manim -qm -v WARNING Grover_3qbit

class Grover_3qbit(Scene):
    config= {
        "x_lim":int,    # define leftmost lowerlimit of x for circuit and gates
        "y_lim":int,    # define up-most limit of y for circuit and gates
        
    }
    def __init__(self, **kwargs):
        super(Grover_3qbit, self).__init__()       #inheritance from class Scene
        self.x_lim=-6.25
        self.y_lim=-1.5
        self.gap=1.5
        self.n_qubit=3
        self.sect=6
        self.x_range=[None for i in range(self.sect+1)]
        self.dots=[Dot() for i in range(self.n_qubit)]
        self.circuitline=[[None for i in range(5)] for i in range(5)]   # List to hold all circuit line elements and VGroup  to include all the Mobjects of circuitline
        self.circuit=VGroup()
    def construct(self):
        
        # Algorithm name
        algorithm="Grover's Algorithm"
        title = Title(algorithm,include_underline=False).set_y(3.1)     #title of the ALgorithm
        self.add_foreground_mobject(title)

        #section zero: create all registers: quantum and classical
        # regs=[MathTex(r"q_"+str(i)) for i in range(n_qubit)] # create quantum register
        
        registers=VGroup()
        regs=[MathTex(r"q_"+str(i)) for i in range(self.n_qubit)]
        for i,reg in enumerate(regs):       #add registers to the scene
            reg.set_x(self.x_lim).set_y(self.y_lim+(i)*self.gap)
            registers.add(reg)
        self.add(registers)

        def create_gate(gate:str,x:int,y:int,control=None,scale=1,textcolor=WHITE,boxcolor=BLUE,fontsize=48,**kwargs):
            if control:
                gate_ = MathTex(gate,color=textcolor,font_size=fontsize).scale(scale+0.5).set_x(x).set_y(y)
                gate_.bg = BackgroundRectangle(gate_,stroke_width=1,color=boxcolor, fill_opacity=1).scale(scale+1)
                if type(control)==list:
                    gate_block=VGroup()
                    gate_block.add(gate_.bg,gate_)
                    for i in range(len(control)):
                        gate_block.add(Dot().set_x(x).set_y(control[i]))
                        gate_block.add(Line([x,control[i],0], [x,y,0]))
                    gate_block.add(gate_.bg,gate_)
                    return gate_block
                else:
                    dot=Dot().set_x(x).set_y(control)
                    control_line= Line([x,control,0], [x,y,0])
                    return VGroup(control_line,gate_.bg,gate_,dot)
            else:
                gate_ = MathTex(gate,color=textcolor,font_size=fontsize).scale(scale+0.5).set_x(x).set_y(y)
                gate_.bg = BackgroundRectangle(gate_,stroke_width=1,color=boxcolor, fill_opacity=1).scale(scale+1)
                return VGroup(gate_.bg,gate_)
        
        def draw_circuitline(sect:int,xrange:list,barrier=True):
            for i in range(self.n_qubit):
                self.circuitline[sect][i]=Line([xrange[0],self.y_lim+i*self.gap,-0.5], [xrange[1],self.y_lim+i*self.gap,-0.5])
                self.circuit.add(self.circuitline[sect][i])
                self.add(self.circuitline[sect][i])
            if barrier:
                barrierline=DashedLine([xrange[1],self.y_lim-0.5,0], [xrange[1],self.y_lim+self.gap*(self.n_qubit-0.5),0],dashed_ratio=0.75).set_opacity(0.5)
                self.add(barrierline)
                # self.add(VGroup(circuit,barrierline))
                # return VGroup(circuit,barrier)
        def move_dot(sect: int,dots=self.dots):
            animation=[]
            for j in range(self.n_qubit):
                animation.append(MoveAlongPath(dots[j], self.circuitline[sect][j]))
            # self.play(*animation)
            return animation
        
        def parallelgates(gate,x,sect:int,qubits=self.n_qubit,scale=1):
            hgates=[]
            for i in range(qubits):
                hgate=create_gate(gate,x=x,y=self.y_lim+i*self.gap,control=None,scale=scale)
                hgates.append(Create(hgate))
            return hgates

        
        # Section I: Uniform Superposition
        depth=sect=1
        self.x_range[0]=[self.x_lim,self.x_lim+0.5+0.75*(depth-1)]
        self.x_range[sect]=[self.x_range[sect-1][1],self.x_range[sect-1][1]+1+0.5*(depth)]
        
        draw_circuitline(sect,xrange=self.x_range[sect])
        annot=MathTex(r"INIT").scale(0.75).set_x(self.x_range[sect][0]+1).set_y(self.y_lim-1)
        self.play(*move_dot(1),*parallelgates(gate=r"H",sect=1,x=1.01*self.x_range[sect][0]+0.75,scale=0.75),Create(annot))

        # # section II: Oracle function Controlled Phase rotation
        sect=2
        depth==0
        self.x_range[sect]=[self.x_range[sect-1][1],self.x_range[sect-1][1]+2+0.5*(depth)]
        pgate_1=create_gate(r"Z",x=self.x_range[sect][0]+(0.5+0.5)*0.75,y=self.y_lim+(2+0.01)*self.gap,
                                        fontsize=48,control=self.y_lim-(0.0)*self.gap,scale=0.75)
        pgate_2=create_gate(r"Z",x=self.x_range[sect][0]+(2+0.5)*0.75,y=self.y_lim+(2+0.01)*self.gap,
                                        fontsize=48,control=self.y_lim+(1.001)*self.gap,scale=0.75)
        draw_circuitline(sect,xrange=self.x_range[sect])
        annot=MathTex(r"Oracle").scale(0.75).set_x(self.x_range[sect][0]+1).set_y(self.y_lim-1)
        self.play(*move_dot(sect),Create(pgate_1), Create(pgate_2),Create(annot))
        print(self.x_range) 
        # Section III: Reflection
        sect=3
        depth=5
        self.x_range[sect]=[self.x_range[sect-1][1],self.x_range[sect-1][1]+1.5+1*(depth+1.25)]
        pgate_1=create_gate(gate=r"Z",x=self.x_range[sect][0]+3.5+0.5*0.75,y=self.y_lim+2.001*self.gap,scale=0.75,
                                        control=[self.y_lim-0*self.gap,self.y_lim+1.0*self.gap])
        draw_circuitline(sect,xrange=self.x_range[sect],barrier=False)
        annot=MathTex("Amplification \; layer").scale(0.75).set_x(sum(self.x_range[sect])/len(self.x_range[sect])).set_y(self.y_lim-1)
        self.play(*move_dot(sect),*parallelgates(gate=r"H",sect=sect,x=self.x_range[sect][0]+0.5+0.5*0.75,scale=0.75),
                                *parallelgates(gate=r"X",sect=sect,x=self.x_range[sect][0]+2.+0.5*0.75,scale=0.75),
                                Create(pgate_1),
                                *parallelgates(gate=r"X",sect=sect,x=self.x_range[sect][0]+5.+0.5*0.75,scale=0.75),
                                *parallelgates(gate=r"H",sect=sect,x=self.x_range[sect][0]+6.5+0.5*0.75,scale=0.75),
                                Create(annot),runtime=3)


        self.wait(2)

        

[[-6.25, -5.75], [-5.75, -4.25], [-4.25, -1.75], None, None, None, None]


In [29]:
dots=[1,2,6]
def test(dots):
    return [dots[i] for i in range(len(dots))]
print(test(dots))

[1, 2, 6]


In [35]:
type(dots)==list

True